In [5]:
%pip install langchain langchain-openai langchain_chroma langchain_community langchainhub beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()
 
#os.environ["SECRET"] = getpass.getpass()
 
token = os.getenv("API_KEY")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"
 
from langchain_openai import ChatOpenAI
 
llm = ChatOpenAI(model=model, base_url=endpoint, openai_api_key=token)

In [6]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# Load, chunk and index the contents of the blog.
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lt.wikipedia.org/wiki/Vilnius",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            #class_=("post-content", "post-title", "post-header")
            id="bodyContent"
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
splits = text_splitter.split_documents(docs)

# print(splits)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(
    model="text-embedding-3-large",
    base_url="https://models.inference.ai.azure.com",
    api_key=token,
))
 


InvalidArgumentError: Collection expecting embedding with dimension of 3072, got 1536

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2017-06-21-overview/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())


In [9]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


c:\Users\sauli\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
def format_docs(docs):
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)
 
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
rag_chain.invoke("What you know about Vilnius?")


[Document(id='e5c64dad-2141-4c85-bed9-df67701c6126', metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'}, page_content='An Overview of Deep Learning for Curious People\n    \nDate: June 21, 2017  |  Estimated Reading Time: 12 min  |  Author: Lilian Weng'), Document(id='a88122de-1ad2-4e4c-b83a-06fc69ca02d3', metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'}, page_content='However, simple perceptron neurons that linearly combine the current input element and the last unit state may easily lose the long-term dependencies. For example, we start a sentence with “Alice is working at …” and later after a whole paragraph, we want to start the next sentence with “She” or “He” correctly. If the model forgets the character’s name “Alice”, we can never know. To resolve the issue, researchers created a special neuron with a much more complicated internal structure for memorizing long-term context, named “Long-short term memory (LSTM)” cell. It

'Vilnius is the capital city of Lithuania, known for its historic old town and vibrant culture. The context provided does not contain information about Vilnius. Therefore, I do not know specific details about Vilnius based on the provided context.'